In [1]:
#Импортирование библиотек
from Timage import Timage
import numpy as np
import scipy.io as sc
from scipy.signal import convolve2d
from PIL import Image

m = sc.loadmat('resources/sector_one.mat')

#Берет n-ое изображение из sector_one.mat
def get_timage(n):
    global m
    return Timage(array=np.array([[m['A'][j][i][n] for j in range(416)] for i in range(442)], dtype=np.float32))

Возьмём 60-е изображение из sector_one.mat и укажем палитру вывода, в которой синий отображает 0, а красный отображает 255

In [ ]:
thermogram = get_timage(60)

pallete = [[0, 0, 255], [255, 0, 0]]

thermogram.show(pallete, contrast_level=0.96)

Обработка по фильтру Гаусса

In [60]:
gausian_blurred = thermogram.gaussian_blur(blur=0.3, radius=9)

gausian_blurred.show(pallete, contrast_level=0.96)

Обработка по медианному фильтру

In [61]:
median_blurred = thermogram.median_blur(radius=1)

median_blurred.show(pallete, contrast_level=0.96)

Шум "соль и перец" и шум по Гауссу 

In [62]:
snp_noised = thermogram.salt_and_pepper_noise(intensity=0.01)

snp_noised.show(pallete, contrast_level=0.96)

In [63]:
gaussian_noised = thermogram.gaussian_noise(mean=0, stddev=0.3)

gaussian_noised.show(pallete, contrast_level=0.96)

Применение медианного фильтра для устранения шумов

In [64]:
snp_noised.median_blur(radius=1).show(pallete, contrast_level=0.96)

In [65]:
gaussian_noised.median_blur(radius=1).show(pallete, contrast_level=0.96)

In [20]:
get_timage(60).gaussian_noise(stddev=2).gaussian_blur(blur=2,radius=10).show(pallete=[[0, 0, 255], [255, 0, 0]], contrast_level=0.94)

In [23]:
get_timage(60).salt_and_pepper_noise(0.01).median_blur().show(pallete=[[0,0,255],[255,0,0]], contrast_level=0.94)

In [28]:
t = get_timage(15)
t.show()
im = t.defect_map(radius=3, stddev=1.5, contrast_level=0.998, direction=np.array([-3**0.5/2, 1/2], dtype=np.float32))

Convolving...: 100%|██████████| 436/436 [00:06<00:00, 71.36it/s]


In [8]:
class DefectMap:
    def __init__(self, array):
        self.array = array
    
    def direct(self, direction, contrast_level, color):
        
        directed = 128 + np.sum(self.array * direction, (2))

        mean = np.mean(directed)
        f = lambda x: (x - mean) / (1 - contrast_level) + mean
        contrasted = directed.copy()

        m, n = len(directed), len(directed[0])
        for i in range(m):
            for j in range(n):
                contrasted[i][j] = min(255, max(0,   f(float(directed[i][j]))  ))

        colored = np.multiply.outer(contrasted.copy(), np.array(color, dtype=np.float32)/255)
        return colored

In [9]:
from math import sin, cos, pi
from tqdm import trange
t60 = get_timage(60)
def_map = DefectMap(array=t60.defect_map(radius=3, stddev=1.5))
num_pictures = 1200
# 1200 -> 14min, 6min
for i in trange(num_pictures):
    phi = i * pi / (num_pictures - 1) # [0, pi]
    im = Image.fromarray(def_map.direct(np.array([cos(phi), sin(phi)], dtype=np.float32), contrast_level=0.998, color=np.array([255,255,255], dtype=np.float32)).astype('uint8'))
    im.save(f'60rotate_meme/{i}.png')

100%|██████████| 1200/1200 [06:49<00:00,  2.93it/s]


In [ ]:
from tqdm import trange
for i in trange(1000):
    get_timage(i).defect_map(radius=3, stddev=1.5, contrast_level=0.998, direction=np.array([2**(-0.5), 2**(-0.5)], dtype=np.float32), color=np.array([255,255,255], dtype=np.float32))

In [ ]:
from moviepy import ImageSequenceClip

output_video = "60rotate_result1200.mp4"
fps = 240

image_files = [f'60rotate_meme//{i}.png' for i in range(1200)]

clip = ImageSequenceClip(image_files, fps=fps)

clip.write_videofile(output_video, audio=False)

MoviePy - Building video 60rotate_result1200.mp4.
MoviePy - Writing video 60rotate_result1200.mp4



MoviePy - Done !
MoviePy - video ready 60rotate_result1200.mp4
